In [1]:
import glob
import os
import re
import pandas as pd

In [2]:
aa3_to_aa1 = {
    "ALA":"A","ARG":"R","ASN":"N","ASP":"D","CYS":"C",
    "GLN":"Q","GLU":"E","GLY":"G","HIS":"H","ILE":"I",
    "LEU":"L","LYS":"K","MET":"M","PHE":"F","PRO":"P",
    "SER":"S","THR":"T","TRP":"W","TYR":"Y","VAL":"V"
}

def to_short_label(s: str) -> str:
    s = str(s).strip()
    s = re.sub(r"\s+", "", s)
    m = re.match(r"^([A-Za-z]{3})(\d+)", s)
    if not m:
        return s
    aa3 = m.group(1).upper()
    resnum = m.group(2)
    return f"{aa3_to_aa1.get(aa3, aa3)}{resnum}"

def parse_res(label: str):
    label = str(label).strip()
    m = re.match(r"^([A-Za-z]+)(\d+)$", label)
    if m:
        return (m.group(1).upper(), int(m.group(2)))
    return (label.upper(), 10**12)

def prefer_DE_first(a: str, b: str):
    aa1, n1 = parse_res(a)
    aa2, n2 = parse_res(b)

    def pri(aa):
        if aa in ("D", "E"):
            return 0
        if aa in ("K", "R"):
            return 1
        return 2

    k1 = (pri(aa1), n1, aa1)
    k2 = (pri(aa2), n2, aa2)
    return (a, b) if k1 <= k2 else (b, a)

def ed_block_priority(resA_label: str):
    """
    You want: E block first, then D block.
    resA is the first residue in 'ResA-ResB' after prefer_DE_first().
    """
    aa, _ = parse_res(resA_label)
    if aa == "E":
        return 0
    if aa == "D":
        return 1
    return 2  # anything else last

input_files = glob.glob("*_details.dat")

for file_path in input_files:
    df = pd.read_csv(file_path, delim_whitespace=True, header=None)

    # skip first 2 header lines
    df = df.iloc[2:].reset_index(drop=True)

    # clean residue columns
    df[0] = df[0].astype(str).str.strip().str.replace(r"\s+", "", regex=True)
    df[1] = df[1].astype(str).str.strip().str.replace(r"\s+", "", regex=True)

    # occupancy -> integer (values preserved)
    occ = (
        df[2].astype(str).str.strip()
        .str.replace("%", "", regex=False)
        .astype(float)
        .round(0)
        .astype(int)
    )

    # filter
    mask = (
        (occ >= 20) &
        (~df[0].str.contains("-Main", na=False)) &
        (~df[1].str.contains("-Main", na=False))
    )

    df_f = df[mask].reset_index(drop=True)
    occ_f = occ[mask].reset_index(drop=True)

    # convert residues to short labels
    res1 = df_f[0].apply(to_short_label)
    res2 = df_f[1].apply(to_short_label)

    # force D/E first in display (E31-K21 etc.)
    canon = [prefer_DE_first(a, b) for a, b in zip(res1, res2)]
    resA = [x[0] for x in canon]
    resB = [x[1] for x in canon]

    out_df = pd.DataFrame({
        "ResA": resA,
        "ResB": resB,
        "Occupancy (%)": occ_f.values
    })

    # remove empties / duplicates
    out_df = out_df.dropna()
    out_df = out_df[(out_df["ResA"] != "") & (out_df["ResB"] != "")]
    out_df = out_df.drop_duplicates(subset=["ResA", "ResB"])

    # ---- sorting: E-first (numerical), then D-first (numerical) ----
    a_parsed = out_df["ResA"].map(parse_res)
    b_parsed = out_df["ResB"].map(parse_res)

    out_df["_a_aa"] = [x[0] for x in a_parsed]
    out_df["_a_n"]  = [x[1] for x in a_parsed]
    out_df["_b_n"]  = [x[1] for x in b_parsed]

    # primary: E block then D block
    out_df["_ed_block"] = out_df["ResA"].map(ed_block_priority)

    # within each block: sort by ResA number, then by partner number
    out_df = out_df.sort_values(
        by=["_ed_block", "_a_n", "_b_n"],
        ascending=True
    )

    # final output columns
    out_df["Interaction"] = out_df["ResA"] + "-" + out_df["ResB"]
    out_df = out_df[["Interaction", "Occupancy (%)"]]

    base = os.path.splitext(file_path)[0]
    out_path = f"{base}_pasteable_number.csv"

    out_df.to_csv(out_path, index=False, lineterminator="\n")

    print(f"Saved: {out_path}")


Saved: apo_p_4_details_pasteable_number.csv
Saved: holo_1_l_details_pasteable_number.csv
Saved: holo_l_3_details_pasteable_number.csv
Saved: apo_l_2_details_pasteable_number.csv
Saved: apo_p_1_details_pasteable_number.csv
Saved: apo_p_2_details_pasteable_number.csv
Saved: apo_l_4_details_pasteable_number.csv
Saved: holo_p_3_details_pasteable_number.csv
Saved: h_l_1_o_details_pasteable_number.csv
Saved: h_l_1_details_pasteable_number.csv
Saved: apo_l_1_details_pasteable_number.csv
Saved: holo_p_1_details_pasteable_number.csv
Saved: apo_l_3_details_pasteable_number.csv
Saved: holo_p_4_details_pasteable_number.csv
Saved: holo_l_2_details_pasteable_number.csv
Saved: hbonds_details_pasteable_number.csv
Saved: holo_l_1_details_pasteable_number.csv
Saved: holo_p_2_details_pasteable_number.csv
Saved: apo_p_3_details_pasteable_number.csv
Saved: holo_l_4_details_pasteable_number.csv
